In [ ]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import log_loss
!pip install iterative-stratification

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
train_df = pd.read_csv('train_features.csv')
test_df = pd.read_csv('test_features.csv')
drug_df = pd.read_csv('train_drug.csv')
target_df = pd.read_csv('train_targets_scored.csv')

In [ ]:
feature_df = pd.merge(train_df,drug_df, how = 'outer', on = 'sig_id')
feature_df.drop('drug_id',axis=1,inplace=True)

test_id = test_df['sig_id']
test_df.drop('sig_id',axis=1,inplace=True)


In [ ]:
def process(df):
    df['cp_type']=df['cp_type'].map({'trt_cp':0,'ctl_vehicle':1})
    df['cp_dose']=df['cp_dose'].map({'D1':0,'D2':1})
    df['cp_time']=df['cp_time'].map({24:0,48:1,72:2})
    return df

feature_df = process(feature_df)
test_df = process(test_df)

In [ ]:
class MOA(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(875,200)
        #self.dropout1 = nn.Dropout(0.2)
        self.fc2 = nn.Linear(200, 150)
        self.fc3 = nn.Linear(150,100)
        self.fc4 = nn.Linear(100,207)
    def forward(self, x):
        x = self.fc1(x)
        #x = self.dropout1(x)
        x = F.relu(x)     
        x = self.fc2(x)
        x = F.relu(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return F.sigmoid(x)

In [ ]:
model=MOA()
targets=[col for col in target_df.columns]
loss_fn = nn.BCEWithLogitsLoss()

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=0.001)

In [ ]:
test_features_nump=test_df.to_numpy()
x_test_tensors=torch.Tensor(test_features_nump)
pred=model(x_test_tensors)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1944: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


In [ ]:
pred
pred_nump=pred.detach().numpy()
pred_df=pd.DataFrame(pred_nump)